### Creating PDF Reports with Pandas, Jinja and WeasyPrint

Why - 事情變得更加困難的地方是，如果你想把多條數據合併到一個文件中。例如，如果你想把兩個DataFrames放在一張Excel表格中，你需要使用Excel庫來手動構建你的輸出。這當然是可能的，但並不簡單。

What - 本文將介紹一種方法，將多條信息組合成一個HTML模板，然後使用Jinja模板和WeasyPrint將其轉換為一個獨立的PDF文檔。


Pandas在處理大量數據並將其總結為多種文本和視覺表現形式方面非常出色。不費吹灰之力，Pandas就支持輸出到CSV、Excel、HTML、json等。在閱讀本文之前，我建議你先回顧一下之前關於Pandas數據透視表的文章和關於從這些表格生成Excel報告的後續文章。它們解釋了我所使用的數據集以及如何使用數據透視表。

如報告文章所示，使用Pandas將數據輸出到一個Excel文件中的多個工作表，或者從pandas DataFrames中創建多個Excel文件，是非常方便的。然而，如果你想將多條信息合併到一個文件中，並沒有很多簡單的方法可以直接從Pandas中完成。幸運的是，python環境有很多選項可以幫助我們。在這篇文章中，我將使用以下流程來創建一個多頁的PDF文檔。

這種方法的好處是，你可以在這個工作流程中替換你自己的工具。不喜歡Jinja？那就插入mako或你選擇的模板工具。如果你想使用HTML之外的另一種標記類型，就去做吧。

工具
首先，我決定使用HTML作為模板語言，因為它可能是生成結構化數據的最簡單方法，並允許相對豐富的格式。我還認為每個人都知道（或能夠弄清楚）足夠的HTML來生成一個簡單的報告。另外，我也沒有意願去學習一種全新的模板語言。然而，如果你選擇使用其他標記語言，其流程應該是一樣的。

我選擇Jinja是因為我有使用Django的經驗，而且它密切反映了Django的語法。當然還有其他的選擇，所以你可以自由地試驗你的選擇。我認為對於這種方法來說，我們的模板並沒有什麼很複雜的地方，所以任何工具都應該可以使用。

最後，這個工具鏈中最困難的部分是弄清楚如何將HTML渲染成PDF。我覺得目前還沒有一個最佳的解決方案，但我選擇了WeasyPrint，因為它仍然在積極維護，而且我發現我可以比較容易地讓它工作。它的工作有相當多的依賴性，所以我很好奇人們在讓它在Windows上工作時是否有真正的挑戰。作為一個替代方案，我過去曾使用過xhtml2pdf，它也很好用。不幸的是，它的文檔現在還有點缺失，但它已經存在了一段時間，而且確實能有效地從HTML生成PDF。

數據
如上所述，我們將使用我以前文章中的相同數據。為了使這篇文章自成一體，下面是我如何導入數據並生成數據透視表，以及CPU和軟件銷售的平均數量和價格的一些匯總統計。

導入模塊，並讀入銷售漏斗信息。

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
df = pd.read_excel("data/sales-funnel.xlsx")
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [2]:
sales_report = pd.pivot_table(df, 
                              index=["Manager", "Rep", "Product"], 
                              values=["Price", "Quantity"],
                              aggfunc=[np.sum, np.mean], 
                              fill_value=0)
sales_report.head()

sum            mean         
                                         Price Quantity  Price Quantity
Manager      Rep           Product                                     
Debra Henley Craig Booker  CPU           65000        2  32500      1.0
                           Maintenance    5000        2   5000      2.0
                           Software      10000        1  10000      1.0
             Daniel Hilton CPU          105000        4  52500      2.0
                           Software      10000        1  10000      1.0

理想情況下，我們現在想做的是將我們的數據按經理劃分，並在一個頁面上包括一些總結性的統計數據，以幫助了解個人結果與全國平均水平的比較。

In [3]:
print(df[df["Product"]=="CPU"]["Quantity"].mean())
print(df[df["Product"]=="CPU"]["Price"].mean())
print(df[df["Product"]=="Software"]["Quantity"].mean())
print(df[df["Product"]=="Software"]["Price"].mean())

1.8888888888888888
51666.666666666664
1.0
10000.0


### DataFrame Options 數據框架選項

在我們談論模板之前，我有一個簡短的旁白。對於一些快速和骯髒的需求，有時你需要做的就是複制和粘貼數據。幸運的是，DataFrame有一個to_clipboard()函數，可以將整個DataFrame複製到剪貼板上，然後你可以輕鬆粘貼到Excel中。我發現在某些情況下這是一個非常有用的選項。

我們在模板中稍後將使用的另一個選項是to_html()，它將生成一個包含完全組成的HTML表格的字符串，並應用最小的樣式。

### Templating 模板

Jinja模板非常強大，支持很多高級功能，如沙盒執行和自動轉義，但對這個應用程序來說並不是必須的。然而，當你的報告越來越複雜或者你選擇使用Jinja來製作你的Web應用程序時，這些功能將為你提供良好的服務。

Jinja的另一個很好的特點是它包括多個內置的過濾器，這將使我們能夠以一種在Pandas中難以做到的方式來格式化我們的一些數據。

為了在我們的應用程序中使用Jinja，我們需要做3件事。
- 創建一個模板
- 在模板的上下文中添加變量
- 將模板渲染成HTML

In [4]:
# Here is a very simple template, let’s call it myreport.html :

# <!DOCTYPE html>
# <html>
# <head lang="en">
#     <meta charset="UTF-8">
#     <title>{{ title }}</title>
# </head>
# <body>
#     <h2>Sales Funnel Report - National</h2>
#      {{ national_pivot_table }}
# </body>
# </html>

該代碼的兩個關鍵部分是{{ title }}和{{ national_pivot_table }}。 . 它們本質上是變量的佔位符，我們將在渲染文檔時提供這些變量。

為了填充這些變量，我們需要創建一個Jinja環境並獲得我們的模板。

In [5]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("myreport.html")

在上面的例子中，你可以把模板位置的完整路徑放進去。另一個關鍵部分是創建env . 這個變量是我們傳遞內容給模板的方式。我們創建一個名為template_var的字典，其中包含所有我們要傳遞給模板的變量。

注意這些變量的名字如何與我們的模板相匹配。

In [6]:
template_vars = {"title" : "Sales Funnel Report - National",
                 "national_pivot_table": sales_report.to_html()}

最後一步是在輸出中包含變量的情況下渲染HTML。這將創建一個字符串，我們最終將把它傳遞給我們的PDF創建引擎。

In [7]:
html_out = template.render(template_vars)

### Generate PDF 
PDF創建部分也相對簡單。我們需要做一些導入，並將一個字符串傳遞給PDF生成器。

- https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#

從 Python 3.8 開始，擴展模塊的 DLL 依賴性和在 Windows 上用 ctypes 加載的 DLL 現在被更安全地解決。只有系統路徑、包含 DLL 或 PYD 文件的目錄，以及用 add_dll_directory() 添加的目錄會被搜索，以尋找加載時的依賴性。具體來說，PATH 和當前工作目錄不再被使用，對這些的修改將不再對正常的 DLL 解析產生任何影響。

如果你遵循官方文檔中的安裝指南，那麼下面的例子就可以工作。

這很酷，但它很難看。主要原因是我們沒有對它進行任何樣式設計。我們必須使用的樣式機制是CSS。我決定使用一部分藍圖CSS，使其具有非常簡單的風格，並能與渲染引擎配合使用。

在文章的其餘部分，我將使用blue print的typography.css作為style.css的基礎，如下所示。我喜歡這個css的地方是。
- 它比較小，容易理解
- 它能在PDF引擎中工作，不會出現錯誤和警告
- 它包括基本的表格格式，看起來很體面

- https://pbpython.com/pdf-reports.html